In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import re
import numpy as np
import random

# Suppress specific sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.feature_extraction.text")

In [2]:
# Load the dataset
data = pd.read_csv('C:\\Users\\Bill\\Desktop\\ΑΙ-project\\iphi2802.csv', delimiter='\t', engine='python')

# Display the first few rows to check the data
data.head()

# Filter the dataset for rows where 'region_main_id' is 1683
filtered_data = data[data['region_main_id'] == 1683]

# Extract the 'text' column from the filtered dataset
filtered_texts = filtered_data['text']

# Display the filtered texts
print(filtered_texts)

# Define a text cleaning function
def clean_text(text):
    """Clean and normalize text by removing brackets, numbers, and hyphens while preserving other content."""
    # Remove brackets but keep content inside
    text = re.sub(r'[\[\]]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove hyphens
    text = re.sub(r'-', '', text)
    return text

# Apply the cleaning function to the texts
cleaned_texts = filtered_texts.apply(clean_text)

34                                      [οσον] επαρκ[ει].
41               α[---]υς και [γυ]ν[η] φω[κ]αρια χαιρετε.
42                              [αφ]ροδιτηι συριηι μητρω.
59      αγαθηι τυχηι. δημοκρατην αριστογενους προεδρον...
129     [---------- εν τω] ετει υπ[ερ -----] κατεσκε[υ...
                              ...                        
2643                           αντιλοχος. κρητινη ς μυος.
2653                               ψυχη γυνη καλου χαιρε.
2704    [------------- ο δεινα] αγυαιο[υ --------- ανε...
2708                                  παρθενιος συρισκου.
2729    [--------]αιο[-------] [--- πανταλ]εων τα[----...
Name: text, Length: 101, dtype: object


In [3]:
def ngram_vectorize_for_ga(texts):
    """Vectorizes texts as unigram vectors with TF-IDF for use in a genetic algorithm."""
    kwargs = {
        'ngram_range': (1, 1),       # Use only unigrams
        'dtype': 'float64',          # Set the data type to float64
        'decode_error': 'strict',    # Raise an error on decoding issues
        'analyzer': 'word',          # Analyze words (as opposed to characters)
        'stop_words': None,          # No stop words filtering
        'token_pattern': r'\b\w+\b'  # Include single characters
    }
    
    # Initialize the TfidfVectorizer with the specified parameters
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn the vocabulary from the texts and transform them into feature vectors
    x = vectorizer.fit_transform(texts)
    
    return x.toarray(), vectorizer.get_feature_names_out(), vectorizer

In [4]:
def generate_population(size, word_count):
    """Generate initial population of individuals."""
    return np.array([random.sample(range(word_count), 2) for _ in range(size)])

# Crossover Methods
def single_point_crossover(parent1, parent2):
    """Perform single-point crossover between two parents."""
    point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

def multi_point_crossover(parent1, parent2, num_points=2):
    """Perform multi-point crossover between two parents."""
    num_points = min(num_points, len(parent1) - 1)  # Ensure num_points does not exceed available positions
    if num_points < 1:
        raise ValueError("Number of points for crossover must be at least 1")
    
    points = sorted(random.sample(range(1, len(parent1)), num_points))
    child1, child2 = parent1.copy(), parent2.copy()
    
    for i in range(len(points)):
        if i % 2 == 0:
            child1[points[i]:points[i + 1] if i + 1 < len(points) else None] = parent2[points[i]:points[i + 1] if i + 1 < len(points) else None]
            child2[points[i]:points[i + 1] if i + 1 < len(points) else None] = parent1[points[i]:points[i + 1] if i + 1 < len(points) else None]
    
    return child1, child2

def uniform_crossover(parent1, parent2):
    """Perform uniform crossover between two parents."""
    child1, child2 = parent1.copy(), parent2.copy()
    for i in range(len(parent1)):
        if random.random() < 0.5:
            child1[i], child2[i] = child2[i], child1[i]
    return child1, child2

def mutate(individual, word_count, mutation_rate):
    """Mutate an individual with a given mutation rate."""
    if random.random() < mutation_rate:
        index = random.randint(0, 1)
        individual[index] = random.randint(0, word_count - 1)
    return individual

#Select Methods
def roulette_selection_cost(population, fitness_scores):
    """Roulette selection based on cost (fitness scores)."""
    total_fitness = np.sum(fitness_scores)
    selection_probs = fitness_scores / total_fitness
    return population[np.random.choice(len(population), p=selection_probs)]

def roulette_selection_placement(population, fitness_scores):
    """Roulette selection based on placement (rank-based)."""
    ranks = np.argsort(fitness_scores)
    rank_probs = np.array([1.0 / (rank + 1) for rank in range(len(population))])
    rank_probs /= rank_probs.sum()
    return population[np.random.choice(len(population), p=rank_probs)]

def tournament_selection(population, fitness_scores, tournament_size):
    """Tournament selection."""
    tournament_indices = np.random.choice(len(population), tournament_size)
    tournament = population[tournament_indices]
    tournament_fitness = fitness_scores[tournament_indices]
    winner_index = np.argmax(tournament_fitness)
    return tournament[winner_index]

def find_top_closest_phrases(target_vector, all_vectors, top_n=10):
    """Find the top N closest phrases to the target vector based on cosine similarity."""
    similarities = cosine_similarity(target_vector, all_vectors).flatten()
    closest_indices = similarities.argsort()[-top_n:][::-1]
    return closest_indices, similarities[closest_indices]

def evaluate_phrase(completed_phrase_vector, top_closest_vectors):
    """Evaluate the similarity of a completed phrase with the top closest vectors."""
    average_similarity = cosine_similarity(completed_phrase_vector, top_closest_vectors).mean()
    return average_similarity

def fitness(individual, index_to_word, target_phrase, all_vectors, all_texts_vectorized, vectorizer):
    """Evaluate the fitness of an individual."""
    completed_phrase = target_phrase.replace('[...]', f'{index_to_word[individual[0]]} {index_to_word[individual[1]]}')
    
    # Vectorize the completed phrase using the same vectorizer
    completed_phrase_vector = vectorizer.transform([completed_phrase])
    
    # Find the top 10 closest phrases in the region
    top_indices, _ = find_top_closest_phrases(completed_phrase_vector, all_texts_vectorized, top_n=10)
    top_closest_vectors = all_texts_vectorized[top_indices]
    
    # Evaluate based on cosine similarity with the top 10 closest phrases
    return evaluate_phrase(completed_phrase_vector, top_closest_vectors)


In [5]:
def genetic_algorithm(index_to_word, target_phrase, all_vectors, all_texts_vectorized, vectorizer, 
                     selection_method='roulette_cost', crossover_method='single_point', 
                     elitism_count=5, mutation_rate=0.01, crossover_rate=0.7, 
                     improvement_threshold=0.01, patience=10, 
                     num_generations=100, population_size=50, tournament_size=5):
    """Run the genetic algorithm with elitism, crossover rate, and improvement-based stopping criteria."""
    word_count = len(index_to_word)
    population = generate_population(population_size, word_count)
    
    best_fitness_so_far = -np.inf
    no_improvement_count = 0
    
    for generation in range(num_generations):
        fitness_scores = np.array([fitness(individual, index_to_word, target_phrase, all_vectors, all_texts_vectorized, vectorizer) for individual in population])
        
        # Sort the population based on fitness scores (higher is better)
        sorted_indices = np.argsort(fitness_scores)[::-1]
        sorted_population = population[sorted_indices]
        sorted_fitness_scores = fitness_scores[sorted_indices]
        
        # Select the elites
        elites = sorted_population[:elitism_count]
        elite_scores = sorted_fitness_scores[:elitism_count]
        
        new_population = list(elites)
        while len(new_population) < population_size:
            # Selection
            if selection_method == 'roulette_cost':
                parent1 = roulette_selection_cost(population, fitness_scores)
                parent2 = roulette_selection_cost(population, fitness_scores)
            elif selection_method == 'roulette_placement':
                parent1 = roulette_selection_placement(population, fitness_scores)
                parent2 = roulette_selection_placement(population, fitness_scores)
            elif selection_method == 'tournament':
                parent1 = tournament_selection(population, fitness_scores, tournament_size=tournament_size)
                parent2 = tournament_selection(population, fitness_scores, tournament_size=tournament_size)
            else:
                raise ValueError(f"Unknown selection method: {selection_method}")

            # Apply crossover based on rate
            if random.random() < crossover_rate:
                if crossover_method == 'single_point':
                    child1, child2 = single_point_crossover(parent1, parent2)
                elif crossover_method == 'multi_point':
                    child1, child2 = multi_point_crossover(parent1, parent2)
                elif crossover_method == 'uniform':
                    child1, child2 = uniform_crossover(parent1, parent2)
                else:
                    raise ValueError(f"Unknown crossover method: {crossover_method}")
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Apply mutation
            new_population.append(mutate(child1, word_count, mutation_rate))
            new_population.append(mutate(child2, word_count, mutation_rate))
        
        # Ensure the new population size matches the original population size
        if len(new_population) > population_size:
            new_population = new_population[:population_size]
        
        population = np.array(new_population)
        
        current_best_fitness = fitness_scores.max()
        print(f"Generation {generation + 1}: Best fitness {current_best_fitness}")
        
        # Check improvement
        if current_best_fitness - best_fitness_so_far < improvement_threshold * best_fitness_so_far:
            no_improvement_count += 1
        else:
            no_improvement_count = 0
            best_fitness_so_far = current_best_fitness
        
        if no_improvement_count >= patience:
            print(f"Stopping early at generation {generation + 1} due to insufficient improvement.")
            break
    
    # Return the best individual found
    return population[np.argmax(fitness_scores)], best_fitness_so_far, generation + 1

In [6]:
def run_experiments(num_experiments, **kwargs):
    """Run the genetic algorithm multiple times and compute average results."""
    all_best_fitness = []
    all_best_individuals = []
    all_generations = []

    for _ in range(num_experiments):
        best_individual , best_fitness, generations = genetic_algorithm(**kwargs)
        all_best_fitness.append(best_fitness)
        all_generations.append(generations)
        all_best_individuals.append(best_individual)

    avg_best_fitness = np.mean(all_best_fitness)
    avg_generations = np.mean(all_generations)

    # Find the best individual from all experiments
    best_experiment_index = np.argmax(all_best_fitness)
    best_individual = all_best_individuals[best_experiment_index]
    best_fitness = all_best_fitness[best_experiment_index]

    # Map indices to words
    index_to_word = params['index_to_word']
    best_words = [index_to_word[i] for i in best_individual]
    
    print(f"Best individual: {best_individual}")
    print(f"Best individual (words): {best_words}")
    print(f"Best fitness: {best_fitness}")
    
    return avg_best_fitness, avg_generations

In [7]:
# Parameters
NUM_GENERATIONS = 1000
POPULATION_SIZE = 200
MUTATION_RATE = 0.01
TOURNAMENT_SIZE = 20
CROSSOVER_RATE =0.1

# Vectorize the texts
vectorized_texts, feature_names, vectorizer = ngram_vectorize_for_ga(filtered_texts)

# Create a mapping from index to word
index_to_word = {index: word for index, word in enumerate(feature_names)}

# Display the resulting TF-IDF matrix and feature names
print("Vectorized Texts (TF-IDF matrix):")
print(vectorized_texts)

print("\nFeature Names:")
print(feature_names)

# Define the target phrase with placeholders
target_phrase = '[...] αλεξανδρε ουδις [...]'

# Define parameters
params = {
    'index_to_word': index_to_word,  
    'target_phrase': target_phrase,
    'all_vectors': vectorized_texts,
    'all_texts_vectorized': vectorized_texts,
    'vectorizer': vectorizer, 
    'selection_method': 'tournament',  # Example selection method
    'crossover_method': 'uniform',  # Example crossover method
    'elitism_count': 20,
    'mutation_rate': MUTATION_RATE,
    'crossover_rate': CROSSOVER_RATE,
    'improvement_threshold': 0.01,
    'patience': 20,
    'num_generations': NUM_GENERATIONS,
    'population_size': POPULATION_SIZE,
    'tournament_size': TOURNAMENT_SIZE
}

# Run the genetic algorithm 10 times and get average results
avg_best_fitness, avg_generations = run_experiments(num_experiments=10, **params)

print(f"Average best fitness score: {avg_best_fitness}")
print(f"Average number of generations: {avg_generations}")

Vectorized Texts (TF-IDF matrix):
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.21189854 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.15175965 0.         ... 0.         0.         0.        ]]

Feature Names:
['0' 'α' 'αγ' ... 'ωσπερ' 'ωτον' 'ωτος']
Generation 1: Best fitness 0.10558996813129382
Generation 2: Best fitness 0.18690658263310536
Generation 3: Best fitness 0.18690658263310536
Generation 4: Best fitness 0.18690658263310536
Generation 5: Best fitness 0.18690658263310536
Generation 6: Best fitness 0.18690658263310536
Generation 7: Best fitness 0.18690658263310536
Generation 8: Best fitness 0.18690658263310536
Generation 9: Best fitness 0.18690658263310536
Generation 10: Best fitness 0.186906582633